# 🚍 Crowd Counting Model Training
This notebook trains CSRNet and P2PNet models on the ShanghaiTech dataset using a free GPU.

**Instructions:**
1. Go to Runtime > Change runtime type > Select **T4 GPU**
2. Upload the dataset zip (instructions below)
3. Run all cells

In [ ]:
# Check GPU
!nvidia-smi
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## Step 1: Mount Google Drive & Upload Dataset
Upload your `shanghaitech.zip` to Google Drive first.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip dataset (adjust path if needed)
!unzip -q /content/drive/MyDrive/shanghaitech.zip -d /content/data/
!ls /content/data/

## Step 2: Install Dependencies

In [ ]:
!pip install scipy h5py tqdm ultralytics

## Step 3: CSRNet Model & Training

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()
        vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.frontend = nn.Sequential(*list(vgg.features.children())[:23])
        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2),
            nn.ReLU(inplace=True),
        )
        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x

print("CSRNet model defined!")

In [ ]:
import os
from PIL import Image
import h5py
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CrowdDataset(Dataset):
    def __init__(self, root_path):
        self.root = root_path
        self.img_dir = os.path.join(root_path, 'images')
        self.gt_dir = os.path.join(root_path, 'ground_truth_h5')
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.endswith('.jpg')])
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        gt_name = img_name.replace('.jpg', '.h5')
        gt_path = os.path.join(self.gt_dir, gt_name)
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        with h5py.File(gt_path, 'r') as f:
            density = np.array(f['density'])
        density = torch.tensor(density, dtype=torch.float32).unsqueeze(0)
        return img, density

print("Dataset class defined!")

In [ ]:
# Generate density maps if not present
import scipy.io as sio
from scipy.ndimage import gaussian_filter

def generate_density_maps(root):
    for split in ['train_data', 'test_data']:
        split_path = os.path.join(root, split)
        img_dir = os.path.join(split_path, 'images')
        gt_dir = os.path.join(split_path, 'ground_truth')
        h5_dir = os.path.join(split_path, 'ground_truth_h5')
        os.makedirs(h5_dir, exist_ok=True)
        
        images = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
        for img_name in images:
            mat_name = 'GT_' + img_name.replace('.jpg', '.mat')
            mat_path = os.path.join(gt_dir, mat_name)
            h5_path = os.path.join(h5_dir, img_name.replace('.jpg', '.h5'))
            
            if os.path.exists(h5_path):
                continue
            
            img = Image.open(os.path.join(img_dir, img_name))
            w, h = img.size
            mat = sio.loadmat(mat_path)
            points = mat['image_info'][0][0][0][0][0]
            
            density = np.zeros((h//8, w//8), dtype=np.float32)
            for pt in points:
                x, y = int(pt[0]/8), int(pt[1]/8)
                if 0 <= x < w//8 and 0 <= y < h//8:
                    density[y, x] += 1
            density = gaussian_filter(density, sigma=3)
            
            with h5py.File(h5_path, 'w') as f:
                f.create_dataset('density', data=density)
        print(f"Done: {split}")

# Run this (adjust path to your dataset location)
generate_density_maps('/content/data/shanghaitech/part_B_final')
print("Density maps ready!")

In [ ]:
# Train CSRNet
from tqdm import tqdm

device = torch.device('cuda')
print(f"Training on: {device}")

root = '/content/data/shanghaitech/part_B_final'
train_set = CrowdDataset(os.path.join(root, 'train_data'))
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
val_set = CrowdDataset(os.path.join(root, 'test_data'))
val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

model = CSRNet().to(device)
criterion = nn.MSELoss(reduction='sum').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

epochs = 50  # Reduced for faster training
best_mae = float('inf')

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for img, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        img, target = img.to(device), target.to(device)
        output = model(img)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    mae = 0.0
    with torch.no_grad():
        for img, target in val_loader:
            img, target = img.to(device), target.to(device)
            output = model(img)
            mae += abs(output.sum().item() - target.sum().item())
    mae /= len(val_loader)
    
    print(f"Epoch {epoch+1} - Loss: {train_loss/len(train_loader):.4f}, MAE: {mae:.2f}")
    
    if mae < best_mae:
        best_mae = mae
        torch.save(model.state_dict(), '/content/drive/MyDrive/csrnet_best.pth')
        print(f"Saved best model (MAE: {mae:.2f})")

print(f"\nCSRNet training complete! Best MAE: {best_mae:.2f}")

## Step 4: P2PNet Training

In [ ]:
# P2PNet Model
class P2PNet(nn.Module):
    def __init__(self, num_anchors=256):
        super(P2PNet, self).__init__()
        vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(vgg.features.children())[:23])
        self.reg_head = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 2, 1)
        )
        self.cls_head = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1)
        )

    def forward(self, x):
        features = self.backbone(x)
        reg = self.reg_head(features)
        cls = self.cls_head(features)
        B, _, H, W = reg.shape
        reg = reg.permute(0, 2, 3, 1).reshape(B, H*W, 2)
        cls = cls.permute(0, 2, 3, 1).reshape(B, H*W)
        return reg, cls

print("P2PNet model defined!")

In [ ]:
# P2PNet Dataset
import scipy.io as sio

class P2PDataset(Dataset):
    def __init__(self, root_path, transform=None):
        self.root = root_path
        self.img_dir = os.path.join(root_path, 'images')
        self.gt_dir = os.path.join(root_path, 'ground_truth')
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.endswith('.jpg')])
        self.transform = transform or transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(os.path.join(self.img_dir, img_name)).convert('RGB')
        img = self.transform(img)
        mat_name = 'GT_' + img_name.replace('.jpg', '.mat')
        mat = sio.loadmat(os.path.join(self.gt_dir, mat_name))
        points = mat['image_info'][0][0][0][0][0]
        return img, {'points': torch.tensor(points, dtype=torch.float32)}

def p2p_collate(batch):
    images = torch.stack([b[0] for b in batch])
    targets = [b[1] for b in batch]
    return images, targets

print("P2PNet dataset defined!")

In [ ]:
# Train P2PNet (simplified loss)
p2p_model = P2PNet().to(device)
p2p_optimizer = torch.optim.Adam(p2p_model.parameters(), lr=1e-4)

p2p_train = P2PDataset(os.path.join(root, 'train_data'))
p2p_train_loader = DataLoader(p2p_train, batch_size=2, shuffle=True, collate_fn=p2p_collate)
p2p_val = P2PDataset(os.path.join(root, 'test_data'))
p2p_val_loader = DataLoader(p2p_val, batch_size=2, shuffle=False, collate_fn=p2p_collate)

best_mae = float('inf')
for epoch in range(50):
    p2p_model.train()
    total_loss = 0
    for img, targets in tqdm(p2p_train_loader, desc=f"P2P Epoch {epoch+1}/50"):
        img = img.to(device)
        reg, cls = p2p_model(img)
        # Simplified counting loss
        pred_counts = torch.sigmoid(cls).sum(dim=1)
        gt_counts = torch.tensor([len(t['points']) for t in targets], device=device, dtype=torch.float32)
        loss = nn.functional.mse_loss(pred_counts, gt_counts)
        p2p_optimizer.zero_grad()
        loss.backward()
        p2p_optimizer.step()
        total_loss += loss.item()
    
    # Validation
    p2p_model.eval()
    mae = 0
    with torch.no_grad():
        for img, targets in p2p_val_loader:
            img = img.to(device)
            _, cls = p2p_model(img)
            pred = (torch.sigmoid(cls) > 0.5).sum(dim=1)
            for i, t in enumerate(targets):
                mae += abs(pred[i].item() - len(t['points']))
    mae /= len(p2p_val)
    
    print(f"Epoch {epoch+1} - Loss: {total_loss/len(p2p_train_loader):.4f}, MAE: {mae:.2f}")
    
    if mae < best_mae:
        best_mae = mae
        torch.save(p2p_model.state_dict(), '/content/drive/MyDrive/p2pnet_best.pth')
        print(f"Saved best P2PNet (MAE: {mae:.2f})")

print(f"\nP2PNet training complete! Best MAE: {best_mae:.2f}")

## ✅ Done!
Your trained models are saved to Google Drive:
- `csrnet_best.pth`
- `p2pnet_best.pth`

Download them and copy to your local `models/` folder.